In [0]:
import re
import gensim
import torch.nn as nn
import torch
import numpy as np
import torch.nn.functional as F
from torch.autograd import Variable
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

In [0]:
!pip install -U -q PyDrive
import os  
os.chdir('/content/')  
!git clone https://github.com/ruelj2/Google_drive.git  

from Google_drive.handle import Google_drive  
Gd = Google_drive()  


    100% |████████████████████████████████| 993kB 20.2MB/s 
Cloning into 'Google_drive'...
remote: Enumerating objects: 106, done.
remote: Counting objects: 100% (106/106), done.
remote: Compressing objects: 100% (96/96), done.
remote: Total 106 (delta 32), reused 38 (delta 9), pack-reused 0
Receiving objects: 100% (106/106), 23.33 KiB | 2.59 MiB/s, done.
Resolving deltas: 100% (32/32), done.


In [0]:
!pip install -U -q PyDrive ## you will have install for every colab session
from pydrive.auth import GoogleAuth     
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
Gd = Google_drive()  

In [0]:
json_import = drive.CreateFile({'id':'1Y0LdeAzisVk-2BVnoqY9eR61Ax3R6rKD'})
json_import.GetContentFile('GoogleNews-vectors-negative300.bin')

In [0]:
json_import1 = drive.CreateFile({'id':'1RQfHsaaFnEEz664YLKMxkTysRo8mAevS'})
json_import1.GetContentFile('emoji2vec.bin')

In [0]:
json_import1 = drive.CreateFile({'id':'1DXnuatRulmjPV0T3lQPu8oyDw_2f34pK'})
json_import1.GetContentFile('new_file.txt')

In [0]:
file_ID='1Y0LdeAzisVk-2BVnoqY9eR61Ax3R6rKD'
local_dir='/content'
Gd.load_file(local_dir, file_ID)

GoogleNews-vectors-negative300.bin loaded


In [0]:
file_ID='1RQfHsaaFnEEz664YLKMxkTysRo8mAevS'
local_dir='/content'
Gd.load_file(local_dir,file_ID)

emoji2vec.bin loaded


In [0]:
file_ID='1DXnuatRulmjPV0T3lQPu8oyDw_2f34pK'
Gd.load_file(local_dir,file_ID)

new_file.txt loaded


In [0]:
def read_file(file_name):
	data_list=[]
	with open(file_name,"r") as f:
		for line in f:
			line=line.strip()
			label=' '.join((line[line.find("[")+1:line.find("]")]).replace(',','').split())
			text=line[line.find("'")+1:line.find("')")].strip()
			data_list.append([label,text])
	return data_list

In [0]:
file_name="new_file.txt"
psychExp_txt=read_file(file_name)

In [0]:
w2v=gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin',binary=True)

In [0]:
e2v=gensim.models.KeyedVectors.load_word2vec_format('emoji2vec.bin',binary=True)

In [0]:
##consider n-gram features
def ngram(token,n):
  output=[]
  for i in range(n-1,len(token)):
    ngram=' '.join(token[i-n+1:i+1])
    output.append(ngram)
  return output  

In [0]:
def create_feature(text,nrange=(1,1)):
  text_features=[]
  text=text.lower()
  text_alphanum=re.sub('[^a-z0-9#]',' ',text)
  for n in range(nrange[0],nrange[1]+1):
    text_features += ngram(text_alphanum.split(),n)
  return(text_features)  

In [0]:
def convert_label(item, name): 
    items = list(map(float, item.split()))
    label = ""
    for idx in range(len(items)): 
        if items[idx] == 1: 
            label += name[idx] + " "
    
    return label.strip()


In [0]:
emotions = ["😂", '😱', "😠", "😢", "😒", "😳", "😣"]

X_all = []
Y_all = []
for label, text in psychExp_txt:
    Y_all.append(torch.LongTensor(list(map(float,label.split()))))
    X_all.append(text)

In [0]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X_all,Y_all,test_size=0.2,random_state=123)

In [0]:
label_train=[]
label_test=[]
for i in Y_train:
  label_train.append(torch.nonzero(i).squeeze())
for i in Y_test:
  label_test.append(torch.nonzero(i).squeeze())
label_test=label_test

In [0]:
# build vocabulary 
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(X_train)
vocab=vectorizer.get_feature_names()

In [0]:
sentence_train=[]
for i in X_train:
  b=[]
  i=create_feature(i)
  for j in i:
    try:
      b.append(vocab.index(j))
    except ValueError:
      vocab.append(j)
      b.append(len(vocab)-1)
  sentence_train.append(torch.LongTensor(b))

In [0]:
sentence_test=[]
for i in X_test:
  b=[]
  i=create_feature(i)
  for j in i:
    try:
      b.append(vocab.index(j))
    except ValueError:
      b
  sentence_test.append(torch.LongTensor(b))

In [0]:
weights=np.zeros((len(vocab),300))
for i,word in enumerate(vocab):
  try:
    weights[i]=w2v[word]
  except KeyError:
    weights[i] = np.random.normal(scale=0.6, size=(300, ))
weights=torch.FloatTensor(weights)     

In [0]:
weights_emoji=np.zeros((len(emotions),300))
for i,emoji in enumerate(emotions):
  if emoji in e2v:
    weights_emoji[i]=e2v[emoji]
weights_emoji=torch.FloatTensor(weights_emoji)   

In [0]:
class LSTMClassifier(nn.Module):
  def __init__(self,weight_matrix_of_word,weight_matrix_of_emoji,num_embeddings,hidden_dim,embedding_dim,target_size,batch_size=1):
    super(LSTMClassifier, self).__init__()
    self.hidden_dim=hidden_dim
    self.batch_size=batch_size
    self.embedding_word=nn.Embedding.from_pretrained(weight_matrix_of_word)
    self.embedding_emoji=nn.Embedding.from_pretrained(weight_matrix_of_emoji,freeze=False)
    self.lstm=nn.LSTM(embedding_dim,hidden_dim,batch_first=True)
    self.hidden=self.init_hidden()
    self.linear=nn.Linear(hidden_dim,target_size)
    self.cos=nn.CosineSimilarity(dim=0)
  def init_hidden(self):
    return(Variable(torch.zeros(1,self.batch_size,self.hidden_dim)),Variable(torch.zeros(1,self.batch_size,self.hidden_dim)))
  def forward(self,sentence):
    embeds_word=self.embedding_word(sentence)
    embeds_emoji=(self.embedding_emoji).weight
    x=embeds_word.view(self.batch_size,len(sentence),-1)
    lstm_out,self.hidden=self.lstm(x,self.hidden)
    a=torch.zeros(self.batch_size,len(embeds_emoji))
    output=self.linear(lstm_out[:,-1,:])
    for i,j in enumerate(embeds_emoji):
      for p,q in enumerate(output):
        a[p][i]=self.cos(q,j)
    return a

In [0]:
def accuracy(test_text,test_label,Model,batch_size=1):
  total=len(test_label)
  correct=0
  for sentence,labels in zip(test_text,test_label):
    output=torch.argmax(Model(sentence),dim=1)
    correct += (output==labels).sum()
  return float(correct)/total  

In [0]:
### Train the model
model=LSTMClassifier(weights,weights_emoji,len(vocab),200,300,300)
loss_function=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model.parameters(),lr=0.01,weight_decay=0)
epoch_accuracies = []
best_accuracy=0
for epoch in range(1000):
  i=0
  for sentence,labels in zip(sentence_train,label_train):
    i=i+1
    optimizer.zero_grad()
    model.hidden=model.init_hidden()
    outputs=model(sentence)
    labels=labels.unsqueeze(dim=0)
    loss=loss_function(outputs,labels)
    loss.backward()
    optimizer.step()
    if (i+1)%1000==0:
              print ('Epoch: [%d/%d], Step: [%d/%d] , Loss: %.4f, Dev: %.4f' 
                     % (epoch+1, 1000, i+1, len(sentence_train), 
                        loss.data, accuracy(sentence_train,label_train,model)))
  epoch_accuracies.append(accuracy(sentence_test,label_test,model))
  if epoch_accuracies[-1] > best_accuracy:
    best_accuracy = epoch_accuracies[-1]
    best_epoch = epoch
  if(epoch>=5 and epoch_accuracies[-1]<=epoch_accuracies[-5]):
    break        
  # TODO: implement early stopping here      
  print(best_accuracy, best_epoch)
      


Epoch: [1/1000], Step: [1000/5984] , Loss: 2.0803, Dev: 0.1611
Epoch: [1/1000], Step: [2000/5984] , Loss: 1.9066, Dev: 0.2072
Epoch: [1/1000], Step: [3000/5984] , Loss: 1.8041, Dev: 0.2116
Epoch: [1/1000], Step: [4000/5984] , Loss: 1.9309, Dev: 0.2543
Epoch: [1/1000], Step: [5000/5984] , Loss: 1.6464, Dev: 0.2908
0.28475935828877 0
Epoch: [2/1000], Step: [1000/5984] , Loss: 2.0626, Dev: 0.3341
Epoch: [2/1000], Step: [2000/5984] , Loss: 1.2952, Dev: 0.3336
Epoch: [2/1000], Step: [3000/5984] , Loss: 1.4983, Dev: 0.3314
Epoch: [2/1000], Step: [4000/5984] , Loss: 1.6821, Dev: 0.3690
Epoch: [2/1000], Step: [5000/5984] , Loss: 2.5497, Dev: 0.3852
0.3442513368983957 1
Epoch: [3/1000], Step: [1000/5984] , Loss: 2.0055, Dev: 0.3864
Epoch: [3/1000], Step: [2000/5984] , Loss: 1.1658, Dev: 0.3491
Epoch: [3/1000], Step: [3000/5984] , Loss: 1.3685, Dev: 0.3695
Epoch: [3/1000], Step: [4000/5984] , Loss: 1.6890, Dev: 0.3496
Epoch: [3/1000], Step: [5000/5984] , Loss: 2.2823, Dev: 0.2500
0.3442513368983

In [0]:
def metrics(test_text,test_label,Model,batch_size=1):
  a=np.array(test_label)
  b=np.zeros(len(test_label))
  for i,sentence in enumerate(test_text):
    predict=torch.argmax(Model(sentence))
    b[i]=predict
  F1=f1_score(a,b,average=None)
  F1_total = f1_score(a,b,average='macro')
  rscore=recall_score(a,b,average=None)
  rscore_total=recall_score(a,b,average='macro')
  pscore=precision_score(a,b,average=None)
  pscore_total=precision_score(a,b,average='macro')
  print("F1:"+str(F1))
  print("recall_score:"+str(rscore))
  print("precision_score:"+str(pscore))
  print("F1_toal:"+str(F1_total))
  print("rscore_total:"+str(rscore_total))
  print("pscore_total:"+str(pscore_total))

In [0]:
metrics(sentence_test,label_test,model,batch_size=1)

F1:[0.42590286 0.2877193  0.31799163 0.35616438 0.34343434 0.25657895
 0.27700831]
recall_score:[0.81042654 0.18303571 0.33333333 0.29147982 0.35233161 0.1959799
 0.2293578 ]
precision_score:[0.28885135 0.67213115 0.304      0.45774648 0.33497537 0.37142857
 0.34965035]
F1_toal:0.3235428255596459
rscore_total:0.3422778160587789
pscore_total:0.3969690383289462
